In [ ]:
import socket
from pymodbus.client import ModbusTcpClient

def read_packets_from_file(file_path):
    packets = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the line into a list of integers (hex to int)
            packet = bytes([int(x, 16) for x in line.split()])
            packets.append(packet)
    return packets

def send_modbus_packet(packet, client):
    try:
        # Send the packet using the client's socket
        client.socket.sendall(packet)

        # Receive response
        response = client.socket.recv(1024)
        
        # Check the response length
        if len(response) < 8:
            return "Exception code: Response too short", response
        
        # Check the eighth byte for the exception code
        function_code = packet[7]  # The original function code sent
        exception_code = function_code + 0x80
        
        
        if response[7] == exception_code:
            return f"Exception code: {exception_code}", response
        else:
            return "Normal response", response
    except Exception as e:
        return f"Error: {e}", None

def format_bytes_as_hex(byte_sequence):
    return ' '.join([f'0x{byte:02x}' for byte in byte_sequence])

def main():
    # Modbus TCP server details
    server_ip = '127.0.0.1'
    server_port = 502
    
    # Create Modbus TCP client
    client = ModbusTcpClient(server_ip, port=server_port)
    
    if not client.connect():
        print("Unable to connect to the Modbus TCP server.")
        return
    
    # Read packets from text file
    packets = read_packets_from_file('packets(epoch20).txt')
    total_packets = len(packets)
    normal_response_count = 0

    # Send each packet and print the result
    for packet in packets:
        result, response = send_modbus_packet(packet, client)
        packet_hex = format_bytes_as_hex(packet)
        if response:
            
            response_hex = format_bytes_as_hex(response)
            response_8th_byte_hex = f'0x{response[7]:02x}'
        else:
            response_hex = 'No response'
            response_8th_byte_hex = 'N/A'
        
        if result == "Normal response":
            normal_response_count += 1
        print(f"Packet: {packet_hex}\nResponse: {response_hex}\n8th_byte_response: {response_8th_byte_hex}\nResult: {result}\n")
    
    test_case_reception_rate = normal_response_count / total_packets
    print(normal_response_count)
    print(total_packets)
    
    
    print(f"Test System Reception Rate (TSRR): {test_case_reception_rate * 100:.2f}%")
    # Close the client connection
    client.close()

if __name__ == '__main__':
    main()
